In [ ]:
# load R packages
library(readxl)
packageVersion('readxl')
library(dplyr)
packageVersion('dplyr')
library(stringr)
packageVersion('stringr')
library(fastDummies)
packageVersion('fastDummies')
library(tidyr)
packageVersion('tidyr')
library(lubridate)
packageVersion('lubridate')
library(ggplot2)
packageVersion('ggplot2')

In [ ]:
# set directory
project.dir = '...'
data.dir = '...'
regeps.dir = '...'
raw.rpdr.dir = file.path(regeps.dir, '...')
cleaned.rpdr.dir = file.path(regeps.dir, '...')
Qingwen.data.dir = file.path(regeps.dir, '...')
mets.dir = file.path(regeps.dir, '...')

# Prepare files
- General workflow: Keep only metabolites having missing values less than 30%, then impute remaining missing values with half min (Metabolon claims missing values because of low concentration)
    - Check IQR after imputation, select IQR > 0
    - After imputation, log 10 transformation
    - After transformation, normalizing data with pareto scaling to keep variation

In [ ]:
# load mets data from Metabolon
mets.data <- read_excel(file.path(mets.dir,"DATA TABLES.XLSX"), sheet = "Batch-normalized Data")
dim(mets.data)
head(mets.data)

In [ ]:
# Load ID data
mets.patient.ID <- read_excel(file.path(mets.dir,"DATA TABLES.XLSX"), sheet = "Sample Meta Data")
dim(mets.patient.ID)
head(mets.patient.ID)

In [ ]:
# combine mets and ID into one file
# select variables in mets.patient.ID
mets.patient.ID.1 <- mets.patient.ID %>% select(PARENT_SAMPLE_NAME,
                                                            CLIENT_IDENTIFIER,
                                                            CLIENT_SAMPLE_ID,
                                                            SAMPLE_SHIPMENT)
# merge with mets.data
mets <- merge(mets.data, mets.patient.ID.1, by = "PARENT_SAMPLE_NAME")
dim(mets)
head(mets)

## Filter only discovery

In [ ]:
# check how many samples in each sample shipment
length(which(mets$SAMPLE_SHIPMENT == '1')) # 935 1 means 1st shipment => discovery cohort <935>
length(which(mets$SAMPLE_SHIPMENT == '2')) # 1051 2 means 2nd shipment => valdiation Cohort

# change sample shipment to cohort
mets['Cohort'] <- mets$SAMPLE_SHIPMENT

mets$Cohort[mets$Cohort == '1'] <- 'discovery'
mets$Cohort[mets$Cohort == '2'] <- 'valdiation'

head(mets$Cohort)
head(mets$SAMPLE_SHIPMENT)

In [ ]:
# select discovery project
discovery.mets <- mets %>% filter(Cohort == 'discovery')
dim(discovery.mets)
table(discovery.mets$Cohort)

# Check missing value for each patient and each metabolite

In [ ]:
# relocate columns to front, after PARENT_SAMPLE_NAME
discovery.mets <- discovery.mets %>% relocate(CLIENT_IDENTIFIER, CLIENT_SAMPLE_ID, SAMPLE_SHIPMENT, Cohort,
                                        .after = PARENT_SAMPLE_NAME)
dim(discovery.mets)
head(discovery.mets)

In [ ]:
# add missing value count for each patient
discovery.mets$missing.value <- rowSums(is.na(discovery.mets))
which(colnames(discovery.mets) == '35')
discovery.mets$missing.value.percent <- (discovery.mets$missing.value/ncol(discovery.mets[,-c(1:5)]))*100
summary(discovery.mets$missing.value.percent)

In [ ]:
# relocate columns to front, after PARENT_SAMPLE_NAME
discovery.mets <- discovery.mets %>% relocate(missing.value, missing.value.percent,
                                        .after = Cohort)
dim(discovery.mets)
head(discovery.mets)

### Check missing value of each metabolite

In [ ]:
# check missing values of metabolites
discovery.mets.t <- data.frame(t(discovery.mets)) # transpose data
colnames(discovery.mets.t) <- discovery.mets.t[1,]
discovery.mets.t$CHEM_ID <- row.names(discovery.mets.t)
discovery.mets.t <- discovery.mets.t %>% relocate(CHEM_ID, .before = 'BRIG-00516')

# remove uncessary rows
discovery.mets.t <- discovery.mets.t[-c(1:7),]
head(discovery.mets.t,10)

In [ ]:
# add missing value count for each metabolite
discovery.mets.t$missing.value <- rowSums(is.na(discovery.mets.t))
discovery.mets.t$missing.value.percent <- (discovery.mets.t$missing.value/ncol(discovery.mets.t))*100
summary(discovery.mets.t$missing.value.percent)

In [ ]:
# relocate columns
discovery.mets.t <- discovery.mets.t %>% relocate(missing.value, missing.value.percent, .before = 'BRIG-00516') %>% 
                        arrange(desc(missing.value.percent))
head(discovery.mets.t)

In [ ]:
# count how many metabolite missing more than different cut off
dim(discovery.mets.t %>% filter(missing.value.percent < 10))
dim(discovery.mets.t %>% filter(missing.value.percent < 20))
dim(discovery.mets.t %>% filter(missing.value.percent < 30))
dim(discovery.mets.t %>% filter(missing.value.percent < 40))
dim(discovery.mets.t %>% filter(missing.value.percent < 50))

In [ ]:
# match with metabolie name info
# load metabolite info from Qingwen's QC
discovery.mets.info <- read_excel(file.path(mets.dir,"DATA TABLES.XLSX"), sheet = "Chemical Annotation")
discovery.mets.info$CHEM_ID <- as.character(discovery.mets.info$CHEM_ID)
table(discovery.mets.info$SUPER_PATHWAY)
table(discovery.mets.info$TYPE)

In [ ]:
# merge to have metabolie info
discovery.mets.t.info <- discovery.mets.t %>% left_join(discovery.mets.info, by = 'CHEM_ID')
dim(discovery.mets.t.info)
head(discovery.mets.t.info)

In [ ]:
table(discovery.mets.t.info$SUPER_PATHWAY)

In [ ]:
# filter out unnamed, Xenobiotics, partially characterized molecules
discovery.mets.t.info.filtered <- discovery.mets.t.info[! (discovery.mets.t.info$SUPER_PATHWAY %in% c("", "Partially Characterized Molecules", "Xenobiotics")), ]
discovery.mets.t.info.filtered <- discovery.mets.t.info.filtered[! (discovery.mets.t.info.filtered$TYPE %in% c('UNNAMED')), ]
head(discovery.mets.t.info.filtered)
dim(discovery.mets.t.info.filtered)

In [ ]:
# count how many filtered metabolite missing more than different cut off
dim(discovery.mets.t.info.filtered %>% filter(missing.value.percent < 10))
dim(discovery.mets.t.info.filtered %>% filter(missing.value.percent < 20)) 
dim(discovery.mets.t.info.filtered %>% filter(missing.value.percent < 30))
dim(discovery.mets.t.info.filtered %>% filter(missing.value.percent < 40))
dim(discovery.mets.t.info.filtered %>% filter(missing.value.percent < 50))

In [ ]:
# check sub pathway
table((discovery.mets.t.info.filtered %>% filter(missing.value.percent < 10))$SUPER_PATHWAY)
table((discovery.mets.t.info.filtered %>% filter(missing.value.percent < 20))$SUPER_PATHWAY) 
table((discovery.mets.t.info.filtered %>% filter(missing.value.percent < 30))$SUPER_PATHWAY) 
table((discovery.mets.t.info.filtered %>% filter(missing.value.percent < 40))$SUPER_PATHWAY)
table((discovery.mets.t.info.filtered %>% filter(missing.value.percent < 50))$SUPER_PATHWAY)

## Remove missing values

- remove metabolites with missing values higher than 30%
- imputate remaining missing values with half min value

In [ ]:
dim(discovery.mets)

In [ ]:
# remove columns with more than 30% NA, available in 70%
discovery.mets <- discovery.mets[, which(colMeans(!is.na(discovery.mets)) > 0.7)]
ncol(discovery.mets) # 1186
n_col <- ncol(discovery.mets) # no of columns
head(discovery.mets)

# Imputation

In [ ]:
# impute remaining missing values with half minimum
# load impute function
impute <- function(x){
  x[is.na(x)] <- min(x, na.rm=TRUE)/2 #convert the item with NA to 1/2 min value from the column
  x
}

In [ ]:
head(discovery.mets)

In [ ]:
which(colnames(discovery.mets) == '35')

In [ ]:
# impute remaining missing values with half minimum
imputed.data <- discovery.mets
imputed.data[,-c(1:7)] <- apply(discovery.mets[,-c(1:7)],2,impute) # 2 is column, -c(1:7) means removing unnecessary cols
imputed.data <- as.data.frame(imputed.data) # convert back to data frame
head(imputed.data)

ncol(imputed.data) # 1186
sum(is.na(imputed.data[,-c(1:7)]))

names(which(colSums(is.na(imputed.data))>0)) # all were imputed

In [ ]:
# interquartile range
# load function
iqr <- function(x){
  if(class(x) != "numeric") 1
  else IQR(x)
}

In [ ]:
# interquartile range
imputed.data.iqr.raw <- sapply(imputed.data , iqr) # sapply in column

imputed.data.iqr <- imputed.data[which(sapply(imputed.data , iqr) > 0)] # remove only 1 col (sample shipment)
imputed.data.iqr <- as.data.frame(imputed.data.iqr)
dim(imputed.data.iqr)

### Check IQR

In [ ]:
# check iqr
imputed.data.iqr.t <- data.frame(t(imputed.data.iqr))
colnames(imputed.data.iqr.t) <- imputed.data.iqr.t[1,]
imputed.data.iqr.t$CHEM_ID <- row.names(imputed.data.iqr.t)
imputed.data.iqr.t <- imputed.data.iqr.t %>% relocate(CHEM_ID, .before = 'BRIG-00516')

# remove uncessary rows
imputed.data.iqr.t <- imputed.data.iqr.t[-c(1:6),]
head(imputed.data.iqr.t)

In [ ]:
imputed.data.iqr.t$IQR <- apply(imputed.data.iqr.t,1,IQR)
summary(imputed.data.iqr.t$IQR)
head(imputed.data.iqr.t)

**all IQR is greater than 0**

# Exclude xenobiotics and partial characterized metabolites

In [ ]:
head(discovery.mets.info)
dim(discovery.mets.info)

In [ ]:
# filter out unnamed, Xenobiotics, partially characterized molecules
discovery.mets.info.filtered <- discovery.mets.info[! (discovery.mets.info$SUPER_PATHWAY %in% c("", 
                                                                            "Partially Characterized Molecules", 
                                                                            "Xenobiotics")), ]
discovery.mets.info.filtered <- discovery.mets.info.filtered[! (discovery.mets.info.filtered$TYPE %in% c('UNNAMED')), ]
head(discovery.mets.info.filtered)
dim(discovery.mets.info.filtered)

In [ ]:
table(discovery.mets.info.filtered$TYPE) # no unnamed
table(discovery.mets.info.filtered$SUPER_PATHWAY) # no xenobiotic and unpartial characterized

In [ ]:
which(colnames(imputed.data.iqr) == '35')

In [ ]:
colnames(imputed.data.iqr[,c(1:6)])

In [ ]:
# select mets cols in mets file
selected.cols <- discovery.mets.info.filtered$CHEM_ID
selected.cols <- c(selected.cols, c('PARENT_SAMPLE_NAME','CLIENT_IDENTIFIER', 'CLIENT_SAMPLE_ID',
                                   'Cohort', 'missing.value', 'missing.value.percent'))

imputed.data.iqr.filtered <- imputed.data.iqr[,which(colnames(imputed.data.iqr) %in% selected.cols)]
head(imputed.data.iqr.filtered)
ncol(imputed.data.iqr.filtered) # 768

In [ ]:
# log transformation
log.data <- imputed.data.iqr.filtered
log.data[, -c(1:6)] <- log(imputed.data.iqr.filtered[, -c(1:6)])
head(log.data)

In [ ]:
# normalizing data with pareto scaling
pareto <- function(x) {(x-mean(x, na.rm = T))/sqrt(sd(x, na.rm = T))}
log.data.pareto <- log.data
log.data.pareto[,-c(1:6)] <- apply(log.data.pareto[,-c(1:6)],2,pareto)
log.data.pareto<- as.data.frame(log.data.pareto)
head(log.data.pareto)

In [ ]:
# add M in metabolite cols                                         
colnames(log.data.pareto)[-c(1:6)] <- paste('M', colnames(log.data.pareto)[-c(1:6)], sep = '')
head(log.data.pareto)

In [ ]:
# rename
discovery.mets <- log.data.pareto
dim(discovery.mets) # 935 x 816

# Remove duplicated patients

In [ ]:
# count number of duplicated ID in each sample
count_dup_ID <- table(discovery.mets$CLIENT_SAMPLE_ID)
head(count_dup_ID)

# merge frequency of duplicate into data
discovery.mets.dup.ID <- merge(discovery.mets, count_dup_ID, by.x = "CLIENT_SAMPLE_ID", by.y = "Var1") #1986 x 1736
head(discovery.mets.dup.ID$Freq)
table(discovery.mets.dup.ID$Freq) # 921 unique; 14 repeat 2 times

In [ ]:
# filter unique biobank ID
discovery.mets.unique.ID <- discovery.mets.dup.ID %>% filter (Freq == "1") # 921
head(discovery.mets.unique.ID$Freq)

# filter duplicated ID
discovery.mets.filter.dup.ID <- discovery.mets.dup.ID %>% filter(Freq == "2") # 14
head(discovery.mets.filter.dup.ID$Freq)
select(discovery.mets.filter.dup.ID,CLIENT_SAMPLE_ID,missing.value)

In [ ]:
# remove duplicated samples with higher missing value of metss
discovery.mets.filter.dup.ID.1 <- discovery.mets.filter.dup.ID %>% 
    arrange(CLIENT_SAMPLE_ID, missing.value) %>%
    group_by(CLIENT_SAMPLE_ID) %>% 
    mutate(missing_rank = rank(missing.value)) %>%
    arrange(missing_rank) %>%
    as.data.frame()

select(discovery.mets.filter.dup.ID.1,CLIENT_SAMPLE_ID, missing.value, missing_rank)

discovery.mets.filter.dup.ID.2 <- discovery.mets.filter.dup.ID.1 %>% filter(missing_rank == "1") #7
dim(discovery.mets.filter.dup.ID.2) # 7 x 818

In [ ]:
# combine 921 unique ID and 7 selected ID from duplicated ID file
discovery.mets.final.filter.dup.ID <- bind_rows(discovery.mets.unique.ID, discovery.mets.filter.dup.ID.2)
dim(discovery.mets.final.filter.dup.ID) # 928 x 818

In [ ]:
head(discovery.mets.final.filter.dup.ID)

In [ ]:
# relocate columns to front, after missing.value
discovery.mets.final.filter.dup.ID <- discovery.mets.final.filter.dup.ID %>% relocate(Freq, missing_rank, 
                                                                                .after = missing.value)
dim(discovery.mets.final.filter.dup.ID)
head(discovery.mets.final.filter.dup.ID)

In [ ]:
which(colnames(discovery.mets.final.filter.dup.ID) == 'M35')

In [ ]:
final.discovery.mets <- discovery.mets.final.filter.dup.ID[,-c(2:8)]
dim(final.discovery.mets)
head(final.discovery.mets)